In [1]:
# 引入包
import time

import statsmodels.api as sm 
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from functools import wraps

In [2]:
# 显示一个函数的运行时间
def show_running_time(func):
    @wraps(func)
    def _show_time(*args, **kwargs):
        now = datetime.now()
        result = func(*args, **kwargs)
        print(func.__name__, datetime.now() - now)
        return result
    return _show_time

In [3]:
# 构造通用股票code
def get_commoncode(str1):
    
    str2=str1
    if str1[0]=='0':
        str2=str1+'.SZ'
    elif str1[0]=='6':
        str2=str1+'.SH'
        
    return str2

In [14]:
# 获得股票数据窗口数据
# @show_running_time
def get_window_data(all_stock_data_new, stock_code, event_date, hs300, date_range=(-128,-8), index_column="Indexcd", code_column="Stkcd", date_column="Trddt", real_return_column="Dretwd",index_return_column="Retindex"):
    # 构造window data，hs300_return指大盘回报，stock_return指相同期间内的回报。
    
    # 筛选出对应股票代码的股票日交易数据
#     t1 = datetime.now()
#     stock_data = all_stock_data[all_stock_data[code_column] == stock_code]
    stock_data = all_stock_data_new[stock_code]
#     t2 = datetime.now()
#     print("first:", t2 - t1)
    count = 0
    while event_date not in list(stock_data[date_column]) and count <= 7:
        event_date = event_date + timedelta(days=1)  
        count += 1
    
#     print("second:", datetime.now() - t2)
    StockDateIndex = stock_data[stock_data[date_column] == event_date].index[0]

    stock_data = stock_data.loc[(StockDateIndex+date_range[0]):(StockDateIndex+date_range[1]),:]
#     print(stock_data.shape)
    
    StockDateList = stock_data[date_column]
    if stock_code.endswith(".SH"):
        hs300_return = hs300[(hs300[date_column].isin(StockDateList)) & (hs300[index_column]==1)][index_return_column]
    else:
        hs300_return = hs300[(hs300[date_column].isin(StockDateList)) & (hs300[index_column]==399106)][index_return_column]
    stock_return = stock_data[real_return_column]

#     print(len(hs300_return), len(stock_return))
    return hs300_return, stock_return

In [5]:
# 计算alpha和beta
# @show_running_time
def pre_alpha_beta(all_stock_data, stock_code, event_date, hs300):
    # stock_return = alpha + beta*hs300_return
    hs300_return, stock_return = get_window_data(all_stock_data, stock_code, event_date, hs300)
    x = sm.add_constant(list(hs300_return))
    #线性回归增加常数项 y=kx+b
    y = list(stock_return)
    regr = sm.OLS(y,x)
    res = regr.fit() 
    alpha = res.params[0]
    beta = res.params[1]
    
    return alpha, beta

In [6]:
# 计算超额收益率
# @show_running_time
def cal_single_CAR(all_stock_data, stock_code, event_date, hs300):
    
    Alpha, Beta = pre_alpha_beta(all_stock_data, stock_code, event_date, hs300)
    
    calWindowHs300Ret_Data, calWindowStockRet_Data = get_window_data(all_stock_data, stock_code, event_date, date_range=[-1, 1], hs300=hs300)
    
    fitvalue_y = Alpha + Beta*np.array(calWindowHs300Ret_Data)
    AR_array = calWindowStockRet_Data - fitvalue_y
    single_car = AR_array.sum()

    return single_car

In [7]:
# @show_running_time
def get_crisis_data(path):
    crisis_data = pd.read_excel(path, parse_dates=["披露日期"])
    print("crisis_data is loading successfully")
    print(crisis_data.shape)
    print(crisis_data.head())
    print("============done!=============")
    return crisis_data

In [8]:
# get_crisis_data(r"选择危机事件并划分种类.xlsx")

In [9]:
@show_running_time
def get_index_data(path):
    index_return = pd.read_table(path, parse_dates=["Trddt"], encoding="gbk")
    print("index_data_all is loading successfully")
    print(index_return.shape)
    print(index_return.head())
    print(index_return.dtypes)
    print("============done!=============")
    return index_return

In [10]:
@show_running_time
def get_all_stock_data(path):
    stock = pd.read_table(path,dtype={"Stkcd":str}, encoding="gbk", skiprows=[1,2],  parse_dates=["Trddt"])
    stock['Stkcd']=stock['Stkcd'].apply(get_commoncode)
    print("stock_data is loading successfully")
    print(stock.shape)
    print(stock.head())
    print("============done!=============")
    return stock

In [18]:
# 计算CAR
@show_running_time
def cal_all_CAR(paths, all_stock_data, hs300, crisis_data):
    
    for i in crisis_data.index:
        stock_code = crisis_data.loc[i, "证券代码"]
#         print(stock_code)
        event_date = crisis_data.loc[i, "披露日期"]
        try:
            single_car = cal_single_CAR(all_stock_data, stock_code, event_date, hs300=hs300)
#             print(single_car)
            crisis_data.loc[i, "CAR"] = single_car
        except:
            continue

    return crisis_data

In [12]:
if __name__ == "__main__":
    paths = {"crisis_data": r"选择危机事件并划分种类.xlsx", 
            "index_return": r"TRD_Index.txt",
            "stock_data": r"TRD_Dalyr.txt"}
    
    crisis_data = get_crisis_data(paths["crisis_data"])    
    hs300 = get_index_data(paths["index_return"])
    all_stock_data = get_all_stock_data(paths["stock_data"])


crisis_data is loading successfully
(796, 10)
          证券代码   证券名称 证券类型  事件大类    事件类型      发生日期  事件类型编码 事件梗概  \
312  600397.SH  *ST安煤   股票  经营事件      停产  20161124     NaN  NaN   
14   600408.SH  *ST安泰   股票  红色预警  违规行为公告  20150724     5.0  NaN   
167  600408.SH  *ST安泰   股票  红色预警  违规行为公告  20160421     5.0  NaN   
631  600408.SH  *ST安泰   股票  红色预警  违规行为公告  20171214     5.0  NaN   
738  002188.SZ  *ST巴士   股票  红色预警  立案调查开始  20180426     NaN  NaN   

                                                  事件摘要       披露日期  
312  根据《江西省化解过剩产能领导小组办公室关于做好调整2016年度煤矿关闭退出计划的紧急通知》要... 2016-11-25  
14   因未及时披露公司重大事项,未依法履行其他职责，上海证券交易所于2015-07-24依据相关法... 2015-07-24  
167  因未及时披露公司重大事项，中国证券监督管理委员会于2016-04-20依据相关法规给予:公开... 2016-04-21  
631  因未及时披露公司重大事项,未依法履行其他职责，上海证券交易所于2017-12-13依据相关法... 2017-12-14  
738  中国证券监督管理委员会对公司巴士在线股份有限公司(以下简称 “公司” )于 2018 年 4... 2018-04-27  
============done!=============
index_data_all is loading successfully
(24989, 8)
   Indexcd      Trddt  Daywk  Opnindex   Hiindex   Loindex  Cl

In [15]:
gen = all_stock_data.groupby("Stkcd")
all_stock_data_new = {key:value for key, value in gen}

In [19]:
crisis_data_CAR = cal_all_CAR(paths, all_stock_data_new, hs300, crisis_data)

cal_all_CAR 0:00:47.501320


In [20]:
crisis_data_CAR.to_csv("crisis_data_CAR.csv", index=None,encoding="gbk")

In [21]:
from multiprocessing import Pool
from multiprocessing.dummy import Pool as TheadPool

In [23]:
indexes = list(crisis_data.index)
crisis_data["CAR"] = 0

In [24]:
def deal_one_record(i):
    stock_code = crisis_data.loc[i, "证券代码"]
    print(stock_code)
    event_date = crisis_data.loc[i, "披露日期"]
    try:
        single_car = cal_single_CAR(all_stock_data, stock_code, event_date, hs300=hs300)
        print(single_car)
        crisis_data.loc[i, "CAR"] = single_car
    except:
        crisis_data.loc[i, "CAR"] = np.nan
    print("===========")

In [25]:
pool = TheadPool(4)
results = pool.map(deal_one_record, indexes)
pool.close()
pool.join()

600397.SH000922.SZ
002306.SZ

600696.SH
0.01119751199569818
600321.SH
0.006181292911197893
-0.1419235915325367
600696.SH

600408.SH
-0.07999082044017909
000922.SZ
-0.06794069962646423
600696.SH
0.31884510180208797
600321.SH
0.00563286953163651
000922.SZ
0.015566677774518168
000972.SZ
-0.08264386846368346
600696.SH
0.00563286953163651
000922.SZ
-0.03830850938208799
000803.SZ
0.05452541578265056
600421.SH
-0.3082743884339916
000803.SZ
-0.004843333544065978
600408.SH
0.0951920158711484
600654.SH
-0.09319882938567076
000803.SZ
-0.019604404466246617
600423.SH
-0.30495530954518046
600423.SH
-0.07298993632077841
600423.SH
-0.07092263967510039
600423.SH
-0.045128254245315
002604.SZ
-0.15181810190603967
000982.SZ
-0.013682586909817081
002070.SZ
-0.08865299856373787
600408.SH
-0.08920474772283427
002604.SZ
0.01859353774454873
002188.SZ
-0.11978828671428073
000737.SZ
-0.1365178103027374
600074.SH
-0.02669786118795654
000755.SZ
-0.10486500512259003
600074.SH
-0.14275045886821464
600870.SH
-0.08039

KeyboardInterrupt: 

# 协程尝试1

In [40]:
# import asyncio

In [41]:
# # 计算CAR
# async def cal_all_CAR(paths, all_stock_data, hs300, crisis_data):
    
#     for i in crisis_data.index:
#         stock_code = crisis_data.loc[i, "证券代码"]
#         print(stock_code)
#         event_date = crisis_data.loc[i, "披露日期"]
#         try:
#             single_car = cal_single_CAR(all_stock_data, stock_code, event_date, hs300=hs300)
#             print(single_car)
#             crisis_data.loc[i, "CAR"] = single_car
#         except:
#             continue

#     return crisis_data

In [14]:
# loop = asyncio.get_event_loop()  # 获取一个event_loop
# tasks = [cal_all_CAR(paths, all_stock_data, hs300, crisis_data)]
# loop.run_until_complete(asyncio.gather(*tasks))  # "阻塞"直到所有的tasks完成
# loop.close()

# 尝试2

In [12]:
from concurrent import futures

In [13]:
@show_running_time
def deal_many(indexes: list):
    with futures.ProcessPoolExecutor(3) as e:
        r = e.map(deal_one_record, indexes)
    return r

In [14]:
indexes = list(crisis_data.index)
crisis_data["CAR"] = 0

In [15]:
def deal_one_record(i):
    stock_code = crisis_data.loc[i, "证券代码"]
    print(stock_code)
    event_date = crisis_data.loc[i, "披露日期"]
    try:
        single_car = cal_single_CAR(all_stock_data, stock_code, event_date, hs300=hs300)
        print(single_car)
        crisis_data.loc[i, "CAR"] = single_car
    except:
        crisis_data.loc[i, "CAR"] = np.nan
    print("===========")

In [16]:
deal_many(indexes)

600397.SH
600408.SH
600408.SH
002188.SZ
600408.SH
0.006181292911197893
0.01859353774454873
600074.SH
600074.SH
-0.1365178103027374
600074.SH
-0.004843333544065978
600074.SH
-0.10486500512259003
600074.SH
-0.16609162447669895
600074.SH
-0.10816905544141885
600074.SH
-0.16609162447669895
600074.SH
-0.034434284414482326
600247.SH
-0.043873433311872656
600247.SH
0.0030200894273613756
600247.SH
0.016124021212840317
600247.SH
-0.08865299856373787
600247.SH
600247.SH


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/v_liliting/anaconda2/envs/tensorflow/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-59a647dc0e41>", line 1, in <module>
    deal_many(indexes)
  File "<ipython-input-2-d60248846b38>", line 6, in _show_time
    result = func(*args, **kwargs)
  File "<ipython-input-13-93948d99c5eb>", line 4, in deal_many
    r = e.map(deal_one_record, indexes)
  File "/Users/v_liliting/anaconda2/envs/tensorflow/lib/python3.5/concurrent/futures/_base.py", line 581, in __exit__
    self.shutdown(wait=True)
  File "/Users/v_liliting/anaconda2/envs/tensorflow/lib/python3.5/concurrent/futures/process.py", line 494, in shutdown
    self._queue_management_thread.join()
  File "/Users/v_liliting/anaconda2/envs/tensorflow/lib/python3.5/threading.py", line 1054, in join
    self._wait_for_tstate_lock()
  File "/Users/v_liliting/anaconda2/envs/tenso

KeyboardInterrupt: 

600193.SH
000409.SZ
000585.SZ
002263.SZ
002194.SZ
0.010095166317278914
002194.SZ
-0.030842441188529773
002194.SZ
002194.SZ
000629.SZ
600399.SH
600399.SH
600634.SH
600701.SH
600401.SH
0.1253664372899349
600401.SH
-0.042725949816629305
600401.SH
0.021884466597592068
600401.SH
-0.10751711998923533
002018.SZ
-0.21719758083285967
000693.SZ
-0.0340187687893971
000693.SZ
-0.04855476128779494
000693.SZ
-0.04855476128779494
000693.SZ
000693.SZ
000693.SZ
000693.SZ
000693.SZ
000693.SZ
-0.04855476128779494
000995.SZ
-0.09487843111836321
000995.SZ
-0.04855476128779494
000995.SZ
-0.05720513190903347
000995.SZ
0.005557886542963894
000922.SZ
-0.04855476128779494
000922.SZ
-0.048710715867969004
000922.SZ
-0.04855476128779494
000922.SZ
-0.07999082044017909
000803.SZ
0.00563286953163651
000803.SZ
0.00563286953163651
000803.SZ
-0.03830850938208799
600423.SH
-0.3082743884339916
600423.SH
-0.019604404466246617
600423.SH
-0.30495530954518046
600423.SH
-0.07298993632077841
002604.SZ
-0.07092263967510039
00260

In [40]:
crisis_data.head()

,证券代码,证券名称,证券类型,事件大类,事件类型,发生日期,事件类型编码,事件梗概,事件摘要,披露日期,CAR
312,600397.SH,*ST安煤,股票,经营事件,停产,20161124,NaN,NaN,根据《江西省化解过剩产能领导小组办公室关于做好调整2016年度煤矿关闭退出计划的紧急通知》要...,2016-11-25,0
14,600408.SH,*ST安泰,股票,红色预警,违规行为公告,20150724,5.0,NaN,"因未及时披露公司重大事项,未依法履行其他职责，上海证券交易所于2015-07-24依据相关法...",2015-07-24,0
167,600408.SH,*ST安泰,股票,红色预警,违规行为公告,20160421,5.0,NaN,因未及时披露公司重大事项，中国证券监督管理委员会于2016-04-20依据相关法规给予:公开...,2016-04-21,0
631,600408.SH,*ST安泰,股票,红色预警,违规行为公告,20171214,5.0,NaN,"因未及时披露公司重大事项,未依法履行其他职责，上海证券交易所于2017-12-13依据相关法...",2017-12-14,0
738,002188.SZ,*ST巴士,股票,红色预警,立案调查开始,20180426,NaN,NaN,中国证券监督管理委员会对公司巴士在线股份有限公司(以下简称 “公司” )于 2018 年 4...,2018-04-27,0


# 尝试3

In [12]:
import asyncio
import math

In [13]:
from concurrent.futures import ThreadPoolExecutor

In [14]:
indexes = list(crisis_data.index)
crisis_data["CAR"] = 0

In [16]:
# start = time.time()
# executor = ThreadPoolExecutor(5)

# async def deal_one_record(i):
#     stock_code = crisis_data.loc[i, "证券代码"]
#     print(stock_code)
#     event_date = crisis_data.loc[i, "披露日期"]
#     try:
#         single_car = cal_single_CAR(all_stock_data, stock_code, event_date, hs300=hs300)
#         print(single_car)
#         crisis_data.loc[i, "CAR"] = single_car
#     except:
#         crisis_data.loc[i, "CAR"] = np.nan
#     print("===========")
    
# def sub_loop(indexes):
#     loop = asyncio.new_event_loop()
#     asyncio.set_event_loop(loop)
#     tasks = [deal_one_record(i) for i in indexes]
#     results = loop.run_until_complete(asyncio.gather(*tasks))
    
# async def run(executor, indexes):
#     await asyncio.get_event_loop().run_in_executor(executor, sub_loop, indexes)
    
# def chunks(l, size):
#     n = math.ceil(len(l) / size)
#     for i in range(0, len(l), n):
#         yield l[i:i + n]
        
# event_loop = asyncio.get_event_loop()
# tasks = [run(executor, chunked) for chunked in chunks(indexes, 5)]
# results = event_loop.run_until_complete(asyncio.gather(*tasks))

# print('Use asyncio+aiohttp+ThreadPoolExecutor cost: {}'.format(time.time() - start))

In [16]:
import time

-0.016312656499290902
002069.SZ
-0.01906576368580183
600868.SH
002715.SZ
0.0013226073579301906
002069.SZ
-0.039049187940703584
300208.SZ
-0.05777584194264103
300330.SZ
-0.21969084287386084
002069.SZ
-0.21969084287386084
000767.SZ


In [17]:
2+3

-0.03097013873227865
000767.SZ


5

-0.03097013873227865
000753.SZ
-0.007173416598910605
601633.SH


In [18]:
3+2

-0.0020548934526963686
300208.SZ


5

000782.SZ
-0.02243182051780964
300330.SZ


In [20]:
crisis_data[crisis_data.S]

-0.00755057578062416
600757.SH


,证券代码,证券名称,证券类型,事件大类,事件类型,发生日期,事件类型编码,事件梗概,事件摘要,披露日期,CAR
312,600397.SH,*ST安煤,股票,经营事件,停产,20161124,NaN,NaN,根据《江西省化解过剩产能领导小组办公室关于做好调整2016年度煤矿关闭退出计划的紧急通知》要...,2016-11-25,0.0
14,600408.SH,*ST安泰,股票,红色预警,违规行为公告,20150724,5.0,NaN,"因未及时披露公司重大事项,未依法履行其他职责，上海证券交易所于2015-07-24依据相关法...",2015-07-24,0.0
167,600408.SH,*ST安泰,股票,红色预警,违规行为公告,20160421,5.0,NaN,因未及时披露公司重大事项，中国证券监督管理委员会于2016-04-20依据相关法规给予:公开...,2016-04-21,0.0
631,600408.SH,*ST安泰,股票,红色预警,违规行为公告,20171214,5.0,NaN,"因未及时披露公司重大事项,未依法履行其他职责，上海证券交易所于2017-12-13依据相关法...",2017-12-14,0.0
738,002188.SZ,*ST巴士,股票,红色预警,立案调查开始,20180426,NaN,NaN,中国证券监督管理委员会对公司巴士在线股份有限公司(以下简称 “公司” )于 2018 年 4...,2018-04-27,0.0
344,600074.SH,*ST保千,股票,红色预警,立案调查开始,20161227,NaN,NaN,中国证券监督管理委员会对公司涉嫌信息披露违法违规行为的立案调查开始,2016-12-28,0.0
513,600074.SH,*ST保千,股票,红色预警,违规行为公告,20170812,NaN,NaN,"因未依法履行其他职责,信息披露虚假或严重误导性陈述，中国证券监督管理委员会于2017-08-...",2017-08-12,0.0
625,600074.SH,*ST保千,股票,红色预警,立案调查开始,20171211,NaN,NaN,中国证券监督管理委员会对公司涉嫌信息披露违规违法行为的立案调查开始,2017-12-14,0.0
639,600074.SH,*ST保千,股票,红色预警,立案调查开始,20171219,NaN,NaN,江苏保千里视像科技集团股份有限公司(以下简称“公司”)股东庄敏于2017年12月18日收到中...,2017-12-19,0.0
660,600074.SH,*ST保千,股票,红色预警,违规行为公告,20180104,NaN,NaN,"因信息披露虚假或严重误导性陈述,未依法履行其他职责，上海证券交易所于2018-01-03依据...",2018-01-04,0.0


0.012843610015316788
600175.SH
002715.SZ
0.01028063922907065
600368.SH
0.015469441843830194
000783.SZ
0.013755131610527667
601688.SH
0.0042569924628792265
300081.SZ
0.0021295152795495183
000007.SZ
0.18344121490474927
600036.SH
-0.012413635303191006
000723.SZ
-0.027338792014603288
600036.SH
0.01630007747102534
601688.SH
-0.027338792014603288
600120.SH
-0.04155120240077329
000010.SZ
0.0298774879986345
601688.SH
-0.009070172442033852
600070.SH
0.0012345407177932729
000010.SZ
-0.01701252240721594
002061.SZ
-0.01819118428088993
000687.SZ
0.050108518029192015
603991.SH
0.0053293321079739536
600869.SH
-0.0234162085432852
000010.SZ
300167.SZ
0.00012143550261103572
000519.SZ
-0.04685756992511206
600290.SH
-0.11020244830537154
600970.SH
0.026689599191498957
000010.SZ
0.00323013567024431
600482.SH
-0.03996318135398633
600290.SH
-0.016727314065676017
600730.SH
0.07293339674287432
000010.SZ
-0.07822602355777239
600730.SH
-0.03996318135398633
300071.SZ
0.025451540014837862
600730.SH
0.03301564881882

In [21]:
2

-0.21940437636070273
002496.SZ


2

-0.14981495114717358
002647.SZ
000020.SZ
0.0008640813001315785
600139.SH
300062.SZ
0.16227539313811526
002496.SZ
0.017694665394660976
002647.SZ
0.03678871279465824
600983.SH
0.008416898599166374
002745.SZ
0.1546050407712785
603030.SH
-0.013118014650883893
600983.SH
601166.SH
-0.031800507585836885
002278.SZ
0.021583263153466176
000012.SZ
-0.005093180114630769
300052.SZ
0.02111618901465017
000692.SZ
0.14588783194559324
600139.SH
0.05637354913366441
000504.SZ
-0.041679330181253166
000007.SZ
0.10709709019314974
601718.SH
-0.011156835055228733
600712.SH
-0.055415247328062654
600728.SH
-0.042112833729877584
600857.SH
601166.SH
-0.12153022314942631
002198.SZ
-0.005093180114630769
300052.SZ
0.026624988734519753
600857.SH
-0.023470953580339092
300117.SZ
601377.SH
-0.09819346404641124
300052.SZ
-0.15309204540108082
600810.SH
002711.SZ
-0.060865762087184444
-0.118465459324838
600139.SH
300117.SZ
-0.09148926859458388
600829.SH
0.27201105062753694
600810.SH
-0.06063444409833813
000762.SZ
601377.SH
